# **Libraries**

In [1]:
import h5py
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.model_selection import cross_validate, KFold
from sklearn.ensemble import RandomForestClassifier
from itertools import combinations
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict

In [2]:
test_size    = 0.20
seed         = 9
h5_color     = '/home/operador2020/AI_bases_framework/color_features.h5'
h5_colorLb   = '/home/operador2020/AI_bases_framework/color_features_labels.h5'
h5_texture   = '/home/operador2020/AI_bases_framework/textural_features.h5'
h5_textureLb = '/home/operador2020/AI_bases_framework/textural_features_labels.h5'
scoring      = {'accuracy' : make_scorer(accuracy_score),
                'f1_score' : make_scorer(f1_score, average = 'weighted'), 
                'precision' : make_scorer(precision_score, average = 'weighted'),
                'recall' : make_scorer(recall_score, average = 'weighted')}


# **Load Data**

In [3]:
# import the feature vector and trained labels
h5f_color   = h5py.File(h5_color, 'r')
h5f_colorLb = h5py.File(h5_colorLb, 'r')

color_features_string          = h5f_color['Banana_Ripeness_Color_Features']
color_features_labels_string   = h5f_colorLb['Banana_Ripeness_Color_Features_Labels']

color_features = np.array(color_features_string)
color_features_labels  = np.array(color_features_labels_string)

h5f_color.close()
h5f_colorLb.close()

# verify the shape of the feature vector and labels
print("[STATUS] features shape: {}".format(color_features.shape))
print("[STATUS] labels shape: {}".format(color_features_labels.shape))

print("[STATUS] training started...")

[STATUS] features shape: (1242, 512)
[STATUS] labels shape: (1242,)
[STATUS] training started...


In [4]:
# import the feature vector and trained labels
h5f_texture  = h5py.File(h5_texture, 'r')
h5f_textureLb = h5py.File(h5_textureLb, 'r')

textural_features_string          = h5f_texture['Banana_Ripeness_Textural_Features']
textural_features_labels_string   = h5f_textureLb['Banana_Ripeness_Textural_Features_Labels']

textural_features = np.array(textural_features_string )
textural_features_labels  = np.array(textural_features_labels_string)

h5f_texture.close()
h5f_textureLb.close()

# verify the shape of the feature vector and labels
print("[STATUS] features shape: {}".format(textural_features.shape))
print("[STATUS] labels shape: {}".format(textural_features_labels.shape))

print("[STATUS] training started...")

[STATUS] features shape: (1242, 13)
[STATUS] labels shape: (1242,)
[STATUS] training started...


In [5]:
global_features = np.hstack([color_features, textural_features])
print("[STATUS] features shape: {}".format(global_features.shape))
print("[STATUS] labels shape: {}".format(textural_features_labels.shape))

[STATUS] features shape: (1242, 525)
[STATUS] labels shape: (1242,)


# **Identify X and Y**

In [6]:
X = global_features
Y = color_features_labels

# **Training and Testing**

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                        test_size   = test_size,
                                        random_state = seed)

X_train.shape, X_test.shape

((993, 525), (249, 525))

# **SVM**

In [8]:
linear_param_grid = {
    
        "kernel": ["linear"],
        "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000],
        "decision_function_shape": ['ovo', 'ovr']
}

In [9]:
poly_param_grid = {
    
        "kernel": ["poly"],
        "degree": [2, 3, 4],
        "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000],
        "decision_function_shape": ['ovo', 'ovr']
}

In [23]:
rbf_param_grid = {
        "kernel": ["rbf"],
        "gamma": ["auto", "scale"],
        "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000],
        "decision_function_shape": ['ovo', 'ovr']
}

In [11]:
sigmoid_param_grid = {
        "kernel": ["sigmoid"],
        "gamma": ["auto", "scale"],
        "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000],
        "decision_function_shape": ['ovo', 'ovr']
}

### **Linear Kernel**

In [55]:
linear_grid = GridSearchCV(svm.SVC(), linear_param_grid)
linear_grid.fit(X_train,y_train)
print(linear_grid.best_estimator_)

SVC(C=10, decision_function_shape='ovo', kernel='linear')


In [56]:
linear_pred = linear_grid.predict(X_test)
linear_accuracy = accuracy_score(y_test, linear_pred)
linear_precision = precision_score(y_test, linear_pred, average='weighted')
linear_recall = recall_score(y_test, linear_pred, average='weighted')
linear_f1 = f1_score(y_test, linear_pred, average='weighted')
print('Accuracy (Linear Kernel): ', "%.2f" % (linear_accuracy*100))
print('F1 (Linear Kernel): ', "%.2f" % (linear_f1*100))
print('Precision (Linear Kernel): ', "%.2f" % (linear_precision*100))
print('Recall (Linear Kernel): ', "%.2f" % (linear_recall*100))

Accuracy (Linear Kernel):  99.20
F1 (Linear Kernel):  99.20
Precision (Linear Kernel):  99.21
Recall (Linear Kernel):  99.20


In [57]:
results = cross_validate(estimator=linear_grid, X=X_train, y=y_train, cv=5, scoring=scoring)

print('Accuracy (Linear Kernel): ', "%.2f" % (np.mean(results['test_accuracy'])*100))
print('F1 (Linear Kernel): ', "%.2f" % (np.mean(results['test_f1_score'])*100))
print('Precision (Linear Kernel): ', "%.2f" % (np.mean(results['test_precision'])*100))
print('Recall (Linear Kernel): ', "%.2f" % (np.mean(results['test_recall'])*100))

Accuracy (Linear Kernel):  98.19
F1 (Linear Kernel):  98.19
Precision (Linear Kernel):  98.22
Recall (Linear Kernel):  98.19


### **Polynomial Kernel**

In [58]:
poly_grid = GridSearchCV(svm.SVC(), poly_param_grid)
poly_grid.fit(X_train,y_train)
print(poly_grid.best_estimator_)

SVC(C=1000, decision_function_shape='ovo', kernel='poly')


In [59]:
poly_pred = poly_grid.predict(X_test)
poly_accuracy = accuracy_score(y_test, poly_pred)
poly_precision = precision_score(y_test, poly_pred, average='weighted')
poly_recall = recall_score(y_test, poly_pred, average='weighted')
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))
print('Precision (Polynomial Kernel): ', "%.2f" % (poly_precision*100))
print('Recall (Polynomial Kernel): ', "%.2f" % (poly_recall*100))

Accuracy (Polynomial Kernel):  99.60
F1 (Polynomial Kernel):  99.60
Precision (Polynomial Kernel):  99.60
Recall (Polynomial Kernel):  99.60


In [60]:
results = cross_validate(estimator=poly_grid, X=X_train, y=y_train, cv=6, scoring=scoring)

print('Accuracy (Polynomial Kernel): ', "%.2f" % (np.mean(results['test_accuracy'])*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (np.mean(results['test_f1_score'])*100))
print('Precision (Polynomial Kernel): ', "%.2f" % (np.mean(results['test_precision'])*100))
print('Recall (Polynomial Kernel): ', "%.2f" % (np.mean(results['test_recall'])*100))

Accuracy (Polynomial Kernel):  98.09
F1 (Polynomial Kernel):  98.09
Precision (Polynomial Kernel):  98.13
Recall (Polynomial Kernel):  98.09


### **RBF Kernel**

In [24]:
rbf_grid = GridSearchCV(svm.SVC(probability=True), rbf_param_grid)
rbf_grid.fit(X_train,y_train)
print(rbf_grid.best_estimator_)

SVC(C=10, decision_function_shape='ovo', probability=True)


In [25]:
rbf_pred = rbf_grid.predict(X_test)
rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_precision = precision_score(y_test, rbf_pred, average='weighted')
rbf_recall = recall_score(y_test, rbf_pred, average='weighted')
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))
print('Precision (RBF Kernel): ', "%.2f" % (rbf_precision*100))
print('Recall (RBF Kernel): ', "%.2f" % (rbf_recall*100))

Accuracy (RBF Kernel):  99.60
F1 (RBF Kernel):  99.60
Precision (RBF Kernel):  99.60
Recall (RBF Kernel):  99.60


In [26]:
results = cross_validate(estimator=rbf_grid, X=X_train, y=y_train, cv=10, scoring=scoring)

print('Accuracy (RBF Kernel): ', "%.2f" % (np.mean(results['test_accuracy'])*100))
print('F1 (RBF Kernel): ', "%.2f" % (np.mean(results['test_f1_score'])*100))
print('Precision (RBF Kernel): ', "%.2f" % (np.mean(results['test_precision'])*100))
print('Recall (RBF Kernel): ', "%.2f" % (np.mean(results['test_recall'])*100))

Accuracy (RBF Kernel):  98.39
F1 (RBF Kernel):  98.39
Precision (RBF Kernel):  98.42
Recall (RBF Kernel):  98.39


In [27]:
rbf_pred_prob = cross_val_predict(estimator=rbf_grid, X=X_train, y=y_train, cv=10, method='predict_proba')

### **Sigmoid Kernel**

In [64]:
sigmoid_grid = GridSearchCV(svm.SVC(), sigmoid_param_grid)
sigmoid_grid.fit(X_train,y_train)
print(sigmoid_grid.best_estimator_)

SVC(C=1000, decision_function_shape='ovo', gamma='auto', kernel='sigmoid')


In [65]:
sigmoid_pred = sigmoid_grid.predict(X_test)
sigmoid_accuracy = accuracy_score(y_test, sigmoid_pred)
sigmoid_precision = precision_score(y_test, sigmoid_pred, average='weighted')
sigmoid_recall = recall_score(y_test, sigmoid_pred, average='weighted')
sigmoid_f1 = f1_score(y_test, sigmoid_pred, average='weighted')
print('Accuracy (Sigmoid Kernel): ', "%.2f" % (sigmoid_accuracy*100))
print('F1 (Sigmoid Kernel): ', "%.2f" % (sigmoid_f1*100))
print('Precision (Sigmoid Kernel): ', "%.2f" % (sigmoid_precision*100))
print('Recall (Sigmoid Kernel): ', "%.2f" % (sigmoid_recall*100))

Accuracy (Sigmoid Kernel):  99.20
F1 (Sigmoid Kernel):  99.20
Precision (Sigmoid Kernel):  99.21
Recall (Sigmoid Kernel):  99.20


In [67]:
results = cross_validate(estimator=sigmoid_grid, X=X_train, y=y_train, cv=8, scoring=scoring)

print('Accuracy (Sigmoid Kernel): ', "%.2f" % (np.mean(results['test_accuracy'])*100))
print('F1 (Sigmoid Kernel): ', "%.2f" % (np.mean(results['test_f1_score'])*100))
print('Precision (Sigmoid Kernel): ', "%.2f" % (np.mean(results['test_precision'])*100))
print('Recall (Sigmoid Kernel): ', "%.2f" % (np.mean(results['test_recall'])*100))

Accuracy (Sigmoid Kernel):  97.88
F1 (Sigmoid Kernel):  97.88
Precision (Sigmoid Kernel):  97.95
Recall (Sigmoid Kernel):  97.88


## **DAGSVM**

In [72]:
def predict_class(X, classifiers, distinct_classes, class_pairs):
    results = []
    for x_row in X:

        class_list = list(distinct_classes)

        # After each prediction, delete the rejected class
        # until there is only one class
        while len(class_list) > 1:
            # We start with the pair of the first and last element in the list
            class_pair = (class_list[0], class_list[-1])
            classifier_index = class_pairs.index(class_pair)
            x_row = x_row.reshape(-1,525)
            y_pred = classifiers[classifier_index].predict(x_row)
            
            if y_pred == 1:
                class_to_delete = class_pair[1]
            else:
                class_to_delete = class_pair[0]

            class_list.remove(class_to_delete)

        results.append(class_list[0])
    return np.array(results)

In [73]:
# Create datasets
training_data = []
class_pairs = list(combinations(set(y_train), 2))
for class_pair in class_pairs:
    class_mask = np.where((y_train == class_pair[0]) | (y_train == class_pair[1]))
    y_i = np.where(y_train[class_mask] == class_pair[0], 1, -1)
    training_data.append((X_train[class_mask], y_i))

### **Linear Kernel**

In [74]:
# Train one classifier per class
classifiers = []
for data in training_data:
    linear = svm.SVC(C=10, decision_function_shape='ovo', kernel='linear')
    linear.fit(data[0], data[1])
    classifiers.append(linear)
print(linear)

SVC(C=10, decision_function_shape='ovo', kernel='linear')


In [75]:
DAG_linear_pred = predict_class(X_test, classifiers, set(y_train), class_pairs)

DAG_linear_accuracy = accuracy_score(y_test, DAG_linear_pred)
DAG_linear_precision = precision_score(y_test, DAG_linear_pred, average='weighted')
DAG_linear_recall = recall_score(y_test, DAG_linear_pred, average='weighted')
DAG_linear_f1 = f1_score(y_test, DAG_linear_pred, average='weighted')
print('Accuracy (DAG Linear Kernel): ', "%.2f" % (DAG_linear_accuracy*100))
print('F1 (DAG Linear Kernel): ', "%.2f" % (DAG_linear_f1*100))
print('Precision (DAG Linear Kernel): ', "%.2f" % (DAG_linear_precision*100))
print('Recall (DAG Linear Kernel): ', "%.2f" % (DAG_linear_recall*100))

Accuracy (DAG Linear Kernel):  99.20
F1 (DAG Linear Kernel):  99.20
Precision (DAG Linear Kernel):  99.21
Recall (DAG Linear Kernel):  99.20


In [76]:
results = cross_validate(estimator=linear, X=X_train, y=y_train, cv=5, scoring=scoring)

print('Accuracy (DAG Linear Kernel): ', "%.2f" % (np.mean(results['test_accuracy'])*100))
print('F1 (DAG Linear Kernel): ', "%.2f" % (np.mean(results['test_f1_score'])*100))
print('Precision (DAG Linear Kernel): ', "%.2f" % (np.mean(results['test_precision'])*100))
print('Recall (DAG Linear Kernel): ', "%.2f" % (np.mean(results['test_recall'])*100))

Accuracy (DAG Linear Kernel):  98.09
F1 (DAG Linear Kernel):  98.09
Precision (DAG Linear Kernel):  98.10
Recall (DAG Linear Kernel):  98.09


### **Polynomial Kernel**

In [77]:
# Train one classifier per class
classifiers = []
for data in training_data:
    poly = svm.SVC(C=1000, decision_function_shape='ovo', kernel='poly')
    poly.fit(data[0], data[1])
    classifiers.append(poly)
print(poly)

SVC(C=1000, decision_function_shape='ovo', kernel='poly')


In [78]:
DAG_poly_pred = predict_class(X_test, classifiers, set(y_train), class_pairs)

DAG_poly_accuracy = accuracy_score(y_test, DAG_poly_pred)
DAG_poly_precision = precision_score(y_test, DAG_poly_pred, average='weighted')
DAG_poly_recall = recall_score(y_test, DAG_poly_pred, average='weighted')
DAG_poly_f1 = f1_score(y_test, DAG_poly_pred, average='weighted')
print('Accuracy (DAG Polynomial Kernel): ', "%.2f" % (DAG_poly_accuracy*100))
print('F1 (DAG Polynomial Kernel): ', "%.2f" % (DAG_poly_f1*100))
print('Precision (DAG Polynomial Kernel): ', "%.2f" % (DAG_poly_precision*100))
print('Recall (DAG Polynomial Kernel): ', "%.2f" % (DAG_poly_recall*100))

Accuracy (DAG Polynomial Kernel):  99.60
F1 (DAG Polynomial Kernel):  99.60
Precision (DAG Polynomial Kernel):  99.60
Recall (DAG Polynomial Kernel):  99.60


In [79]:
results = cross_validate(estimator=poly, X=X_train, y=y_train, cv=6, scoring=scoring)

print('Accuracy (DAG Polynomial Kernel): ', "%.2f" % (np.mean(results['test_accuracy'])*100))
print('F1 (DAG Polynomial Kernel): ', "%.2f" % (np.mean(results['test_f1_score'])*100))
print('Precision (DAG Polynomial Kernel): ', "%.2f" % (np.mean(results['test_precision'])*100))
print('Recall (DAG Polynomial Kernel): ', "%.2f" % (np.mean(results['test_recall'])*100))

Accuracy (DAG Polynomial Kernel):  98.19
F1 (DAG Polynomial Kernel):  98.19
Precision (DAG Polynomial Kernel):  98.24
Recall (DAG Polynomial Kernel):  98.19


### **RBF Kernel**

In [80]:
# Train one classifier per class
classifiers = []
for data in training_data:
    rbf = svm.SVC(C=10, decision_function_shape='ovo')
    rbf.fit(data[0], data[1])
    classifiers.append(rbf)
print(rbf)

SVC(C=10, decision_function_shape='ovo')


In [83]:
DAG_rbf_pred = predict_class(X_test, classifiers, set(y_train), class_pairs)

DAG_rbf_accuracy = accuracy_score(y_test, DAG_rbf_pred)
DAG_rbf_precision = precision_score(y_test, DAG_rbf_pred, average='weighted')
DAG_rbf_recall = recall_score(y_test, DAG_rbf_pred, average='weighted')
DAG_rbf_f1 = f1_score(y_test, DAG_rbf_pred, average='weighted')
print('Accuracy (DAG RBF Kernel): ', "%.2f" % (DAG_rbf_accuracy*100))
print('F1 (DAG RBF Kernel): ', "%.2f" % (DAG_rbf_f1*100))
print('Precision (DAG RBF Kernel): ', "%.2f" % (DAG_rbf_precision*100))
print('Recall (DAG RBF Kernel): ', "%.2f" % (DAG_rbf_recall*100))

Accuracy (DAG RBF Kernel):  99.60
F1 (DAG RBF Kernel):  99.60
Precision (DAG RBF Kernel):  99.60
Recall (DAG RBF Kernel):  99.60


In [84]:
results = cross_validate(estimator=rbf, X=X_train, y=y_train, cv=10, scoring=scoring)

print('Accuracy (DAG RBF Kernel): ', "%.2f" % (np.mean(results['test_accuracy'])*100))
print('F1 (DAG RBF Kernel): ', "%.2f" % (np.mean(results['test_f1_score'])*100))
print('Precision (DAG RBF Kernel): ', "%.2f" % (np.mean(results['test_precision'])*100))
print('Recall (DAG RBF Kernel): ', "%.2f" % (np.mean(results['test_recall'])*100))

Accuracy (DAG RBF Kernel):  98.09
F1 (DAG RBF Kernel):  98.09
Precision (DAG RBF Kernel):  98.15
Recall (DAG RBF Kernel):  98.09


### **Sigmoid Kernel**

In [85]:
# Train one classifier per class
classifiers = []
for data in training_data:
    sigmoid = svm.SVC(C=1000, decision_function_shape='ovo', gamma='auto', kernel='sigmoid')
    sigmoid.fit(data[0], data[1])
    classifiers.append(sigmoid)
print(sigmoid)

SVC(C=1000, decision_function_shape='ovo', gamma='auto', kernel='sigmoid')


In [86]:
DAG_sigmoid_pred = predict_class(X_test, classifiers, set(y_train), class_pairs)

DAG_sigmoid_accuracy = accuracy_score(y_test, DAG_sigmoid_pred)
DAG_sigmoid_precision = precision_score(y_test, DAG_sigmoid_pred, average='weighted')
DAG_sigmoid_recall = recall_score(y_test, DAG_sigmoid_pred, average='weighted')
DAG_sigmoid_f1 = f1_score(y_test, DAG_sigmoid_pred, average='weighted')
print('Accuracy (DAG Sigmoid Kernel): ', "%.2f" % (DAG_sigmoid_accuracy*100))
print('F1 (DAG Sigmoid Kernel): ', "%.2f" % (DAG_sigmoid_f1*100))
print('Precision (DAG Sigmoid Kernel): ', "%.2f" % (DAG_sigmoid_precision*100))
print('Recall (DAG Sigmoid Kernel): ', "%.2f" % (DAG_sigmoid_recall*100))

Accuracy (DAG Sigmoid Kernel):  99.20
F1 (DAG Sigmoid Kernel):  99.20
Precision (DAG Sigmoid Kernel):  99.21
Recall (DAG Sigmoid Kernel):  99.20


In [87]:
results = cross_validate(estimator=sigmoid, X=X_train, y=y_train, cv=8, scoring=scoring)

print('Accuracy (DAG Sigmoid Kernel): ', "%.2f" % (np.mean(results['test_accuracy'])*100))
print('F1 (DAG Sigmoid Kernel): ', "%.2f" % (np.mean(results['test_f1_score'])*100))
print('Precision (DAG Sigmoid Kernel): ', "%.2f" % (np.mean(results['test_precision'])*100))
print('Recall (DAG Sigmoid Kernel): ', "%.2f" % (np.mean(results['test_recall'])*100))

Accuracy (DAG Sigmoid Kernel):  97.88
F1 (DAG Sigmoid Kernel):  97.88
Precision (DAG Sigmoid Kernel):  97.95
Recall (DAG Sigmoid Kernel):  97.88


# **Random Forest**

In [12]:
rf_param_grid = {
     'n_estimators': [90, 100, 115, 130],
     'criterion': ['gini', 'entropy'],
     'max_depth' : [4, 5, 6, 7, 8],
     'max_features': ['auto', 'log2', 'sqrt'] 
}

In [28]:
randomF_grid = GridSearchCV(RandomForestClassifier(), rf_param_grid)
randomF_grid.fit(X_train,y_train)
print(randomF_grid.best_estimator_)

RandomForestClassifier(criterion='entropy', max_depth=8)

In [18]:
rf = RandomForestClassifier(criterion='entropy', max_depth=8)
rf.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=8)

In [19]:
random_pred = rf.predict(X_test)

random_accuracy = accuracy_score(y_test, random_pred)
random_precision = precision_score(y_test, random_pred, average='weighted')
random_recall = recall_score(y_test, random_pred, average='weighted')
random_f1 = f1_score(y_test, random_pred, average='weighted')
       
print('Accuracy (Random Forest): ', "%.2f" % (random_accuracy*100))
print('F1 (Random Forest): ', "%.2f" % (random_precision*100))
print('Precision (Random Forest): ', "%.2f" % (random_recall*100))
print('Recall (Random Forest): ', "%.2f" % (random_f1*100))

Accuracy (Random Forest):  98.39
F1 (Random Forest):  98.39
Precision (Random Forest):  98.39
Recall (Random Forest):  98.39


In [20]:
results = cross_validate(estimator=rf, X=X_train, y=y_train, cv=10, scoring=scoring)

print('Accuracy (Random Forest): ', "%.2f" % (np.mean(results['test_accuracy'])*100))
print('F1 (Random Forest): ', "%.2f" % (np.mean(results['test_f1_score'])*100))
print('Precision (Random Forest): ', "%.2f" % (np.mean(results['test_precision'])*100))
print('Recall (Random Forest): ', "%.2f" % (np.mean(results['test_recall'])*100))

Accuracy (Random Forest):  98.09
F1 (Random Forest):  98.09
Precision (Random Forest):  98.14
Recall (Random Forest):  98.09


In [21]:
random_pred_prob = cross_val_predict(estimator=rf, X=X_train, y=y_train, cv=10, method='predict_proba')

In [29]:
# roc curve for classes
fpr = {}
tpr = {}
thresh ={}

n_class = 3

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_train, random_pred_prob[:,i], pos_label=i)
    
# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Class totally green vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class yellow with green tips vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Class yellow with brown spots vs Rest')
plt.title('(c) Random Forest without PCA')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')  